# Corona Cases

This Jupyter notebook contains a bar chart to visualize the corona cases either per country of worldwide. The underlying dataset is provided by [Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19/). Click ```Cell -> Run All``` in the top menu to refresh the dataset.

## Import packages

- After the initial run, comment this line to minimize the execution time
```bash
#!jupyter nbextension enable --py widgetsnbextension
```

In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets

## Prepare dataset

### Data cleansing

- Replaces NA/NaN values
- Drops not required columns (Province/State, Lat, Long)
- Aggregates the corona cases per country
- Calculates the worldwide corona cases
- Adds a status per case (Confirmed, Death, Recovered)

In [3]:
def data_cleansing(df, status):
    df=df.fillna("")
    df=df.drop(["Province/State", "Lat", "Long"], axis=1)
    df=df.rename(columns={"Country/Region": "Country"})
    df=df.groupby(["Country"], sort=True).sum() # Country will become the new index
    df.loc["Worldwide"] = df.sum(numeric_only=True)
    df.insert(0, "Status", status, allow_duplicates=True)
    df=df.reset_index().set_index(["Country", "Status"])
    return df

### Load confirmed cases

- Load confirmed corona cases
- Perform data cleaning
- Index "Country" + "Status"
- Print out preview

In [4]:
df_confirmed=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
df_confirmed=data_cleansing(df_confirmed, "Confirmed")
df_confirmed.head(2)

,,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20
Country,Status,,,,,,,,,,,,,,,,,,,,,
Afghanistan,Confirmed,0,0,0,0,0,0,0,0,0,0,...,5,7,7,7,11,16,21,22,22,22
Albania,Confirmed,0,0,0,0,0,0,0,0,0,0,...,10,12,23,33,38,42,51,55,59,64


### Load death cases

- Load confirmed corona cases
- Perform data cleaning
- Index "Country" + "Status"
- Print out preview

In [5]:
df_death=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv")
df_death=data_cleansing(df_death, "Death")
df_death.head(2)

,,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20
Country,Status,,,,,,,,,,,,,,,,,,,,,
Afghanistan,Death,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Albania,Death,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,2,2


### Load recovered cases

- Load recovered corona cases
- Perform data cleaning
- Index "Country" + "Status"
- Print out preview

In [6]:
df_recovered=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")
df_recovered=data_cleansing(df_recovered, "Recovered")
df_recovered.head(2)

,,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20
Country,Status,,,,,,,,,,,,,,,,,,,,,
Afghanistan,Recovered,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
Albania,Recovered,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculate active cases

- Calculate active corona cases (active = confirmed - death - recovered)
- Index "Country" + "Status"
- Print out preview

In [7]:
index_active = []
data_active = []
for country in df_confirmed.index.get_level_values(0).unique().tolist():
    index_active.append((country, "Active"))
    data_active.append(
        df_confirmed.loc[country,"Confirmed"]
            .subtract(df_death.loc[country,"Death"])
                .subtract(df_recovered.loc[country,"Recovered"]))

df_active = pd.DataFrame(
    data=data_active, 
    columns=df_confirmed.columns, 
    index=pd.MultiIndex.from_tuples(index_active)
).astype(int)
df_active.head(2)

,,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20
Afghanistan,Active,0,0,0,0,0,0,0,0,0,0,...,5,7,7,7,11,16,20,21,21,21
Albania,Active,0,0,0,0,0,0,0,0,0,0,...,10,11,22,32,37,41,50,54,57,62


### Merge all cases

- Merge all data frames via index "Country" + "Status"
- Sort ascending by "Country" and "Status"
- Print out a preview

In [8]:
df=pd.concat([df_confirmed,df_death,df_recovered,df_active]).sort_values(by=["Country", "Status"])
df.head(8)

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  \
Country     Status                                                            
Afghanistan Active           0        0        0        0        0        0   
            Confirmed        0        0        0        0        0        0   
            Death            0        0        0        0        0        0   
            Recovered        0        0        0        0        0        0   
Albania     Active           0        0        0        0        0        0   
            Confirmed        0        0        0        0        0        0   
            Death            0        0        0        0        0        0   
            Recovered        0        0        0        0        0        0   

                       1/28/20  1/29/20  1/30/20  1/31/20  ...  3/10/20  \
Country     Status                                         ...            
Afghanistan Active           0        0        0        0  ...        5   
            Confirmed        0        0        0        0  ...        5   
            Death            0        0        0        0  ...        0   
            Recovered        0        0        0        0  ...        0   
Albania     Active           0        0        0        0  ...       10   
            Confirmed        0        0        0        0  ...       10   
            Death            0        0        0        0  ...        0   
            Recovered        0        0        0        0  ...        0   

                       3/11/20  3/12/20  3/13/20  3/14/20  3/15/20  3/16/20  \
Country     Status                                                            
Afghanistan Active           7        7        7       11       16       20   
            Confirmed        7        7        7       11       16       21   
            Death            0        0        0        0        0        0   
            Recovered        0        0        0        0        0        1   
Albania     Active          11       22       32       37       41       50   
            Confirmed       12       23       33       38       42       51   
            Death            1        1        1        1        1        1   
            Recovered        0        0        0        0        0        0   

                       3/17/20  3/18/20  3/19/20  
Country     Status                                
Afghanistan Active          21       21       21  
            Confirmed       22       22       22  
            Death            0        0        0  
            Recovered        1        1        1  
Albania     Active          54       57       62  
            Confirmed       55       59       64  
            Death            1        2        2  
            Recovered        0        0        0  

[8 rows x 58 columns]

## Visualize dataset

- Create stacked bar chart to visualize recovered, active and death corona cases
- Confirmed corona cases are the sum of recovered, active and death ones (not visualized)
- Add a drop down to swithc beteen corona cases either per country or worldwide
- Hover over a bar to get the exact number of recovered, active or death corona cases per day

In [9]:
country="Worldwide"
fig = go.FigureWidget(data=[
                    go.Bar(name="Recovered", x=df.columns, y=df.loc[country,"Recovered"], marker_color="green"),
                    go.Bar(name="Active", x=df.columns, y=df.loc[country,"Active"], marker_color="red"),
                    go.Bar(name="Death", x=df.columns, y=df.loc[country,"Death"], marker_color="black")
                ],
                layout=go.Layout(plot_bgcolor = "#EEEEEE"))
fig.update_layout(barmode='stack')

dropdown = widgets.Dropdown(
    description='Corona cases:',
    value="Worldwide",
    options=df.index.get_level_values(0).unique().tolist(),
    style={'description_width': 'initial'}
)
def response(change):
    country=dropdown.value
    with fig.batch_update():
            fig.data[0].y = df.loc[country,"Recovered"]
            fig.data[1].y = df.loc[country,"Active"]
            fig.data[2].y = df.loc[country,"Death"]
dropdown.observe(response, names="value")

widgets.VBox([dropdown, fig])